## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [25]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [26]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [27]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [28]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210212.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [29]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,475110,36300,34500,545910,431642,0.790683,176731,2021-02-11
1,Aragón,81045,6300,5400,92745,88297,0.952041,35659,2021-02-11
2,Asturias,83975,6300,4200,94475,78446,0.830336,32946,2021-02-11
3,Baleares,42480,3200,4800,50480,40624,0.804754,16683,2021-02-11
4,Canarias,93570,7000,8900,109470,99316,0.907244,40779,2021-02-11
5,Cantabria,39345,2900,2400,44645,39707,0.889394,17478,2021-02-11
6,Castilla y Leon,177555,13500,9800,200855,178831,0.890349,75433,2021-02-11
7,Castilla La Mancha,120375,9200,8300,137875,117671,0.853461,41690,2021-02-11
8,Cataluña,410120,31400,31800,473320,376519,0.795485,152091,2021-02-11
9,C. Valenciana,207280,15700,20600,243580,214010,0.878603,92433,2021-02-11


..hacemos una copia para hacerle las transformaciones necesarias:

In [30]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [31]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [32]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [33]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [34]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [35]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,475110,36300,34500,545910,431642,0.790683,176731,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,81045,6300,5400,92745,88297,0.952041,35659,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,83975,6300,4200,94475,78446,0.830336,32946,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,42480,3200,4800,50480,40624,0.804754,16683,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,93570,7000,8900,109470,99316,0.907244,40779,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,39345,2900,2400,44645,39707,0.889394,17478,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,177555,13500,9800,200855,178831,0.890349,75433,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,120375,9200,8300,137875,117671,0.853461,41690,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,410120,31400,31800,473320,376519,0.795485,152091,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,207280,15700,20600,243580,214010,0.878603,92433,11/2/21,12/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [36]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [37]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [38]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [39]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,12/2/21,Andalucía,475110,36300,34500,545910,431642,0.790683,176731,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,12/2/21,Aragón,81045,6300,5400,92745,88297,0.952041,35659,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,12/2/21,Asturias,83975,6300,4200,94475,78446,0.830336,32946,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,12/2/21,Baleares,42480,3200,4800,50480,40624,0.804754,16683,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,12/2/21,Canarias,93570,7000,8900,109470,99316,0.907244,40779,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,12/2/21,Cantabria,39345,2900,2400,44645,39707,0.889394,17478,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,12/2/21,Castilla y Leon,177555,13500,9800,200855,178831,0.890349,75433,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,12/2/21,Castilla La Mancha,120375,9200,8300,137875,117671,0.853461,41690,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,12/2/21,Cataluña,410120,31400,31800,473320,376519,0.795485,152091,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,12/2/21,C. Valenciana,207280,15700,20600,243580,214010,0.878603,92433,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [40]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [41]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [42]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,11/2/21,Navarra,39345.0,2900.0,2700.0,44945,38016,0.845834,14003.0,NaN,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
556,11/2/21,País Vasco,107460.0,7900.0,9100.0,124460,84459,0.678604,32897.0,NaN,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
557,11/2/21,Ceuta,4320.0,400.0,400.0,5120,2559,0.499805,963.0,NaN,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
558,11/2/21,Melilla,3150.0,400.0,400.0,3950,2543,0.643797,1082.0,NaN,10/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [43]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [44]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [45]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [46]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,12/2/21,Navarra,39345.0,2900.0,2700.0,44945,39196,0.872088,14536.0,NaN,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
576,12/2/21,País Vasco,107460.0,7900.0,9100.0,124460,89883,0.722184,36125.0,NaN,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
577,12/2/21,Ceuta,4320.0,400.0,400.0,5120,2697,0.526758,963.0,NaN,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
578,12/2/21,Melilla,3150.0,400.0,400.0,3950,2647,0.670127,1083.0,NaN,11/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [47]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [48]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)